In [1]:
# !pip install Beautifulsoup4
# !pip install selenium
# !pip install webdriver_manager
# !pip install pandas

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import urllib.request
import time
import re

import pandas as pd
import pprint

from bs4 import BeautifulSoup

In [ ]:
# 크롬 드라이버 자동 업데이트
from webdriver_manager.chrome import ChromeDriverManager

# 브라우저 꺼짐 방지
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 불필요한 에러 메세지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())

### 다나와 상세페이지에서 테이블 내용을 불러오기

In [52]:
user_agent ="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
chrome_options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(options=chrome_options)

In [53]:
# 크롤링 참고 (헤더)

header = ["name", "price", "link", "company", "product_seq", "image", "form_factor", "interface",
         "protocol", "volume", "memory_type", "nand", "ram_type", "ram_mounted", "sequential_read", "sequential_write",
         "read_iops", "write_iops", "heatsink", "rgbled", "as_year", "support_option", "reg_date", "bookmark"]

In [54]:
save_file = False  # 파일 저장 여부
waiting_sec = 3

# seq 파일 읽기
# category = 112747  # cpu
# category = 112751  # 메인보드
# category = 112752  # 램
# category = 112753  # 그래픽카드
category = 112760  # SSD
# category = 112763  # HDD
# category = 112775  # 케이스
# category = 112777  # 파워
# category = 11236855  # 쿨러

seq_file = f'./seq_{category}.csv'
seq_df = pd.read_csv(seq_file)
len_seq = len(seq_df)
print(len_seq)

# 비트마스킹
m2i_bitmask = ['SATA', 'NVMe', 'PCIe', 'PCIe4.0', 'PCIe5.0']
m2f_bitmask = ['2230', '2242', '2260', '2280', '2580', '22110', '25110']
wl_bitmask = ['무선 LAN', '블루투스', 'M.2 Key-E(모듈별매)']

support_bitmask = ['TRIM', 'GC', 'SLC캐싱', 'S.M.A.R.T', 'DEVSLP', 'ECC', 'HMB']
addon_bitmask = ['AES 암호화', '전용 S/W']

as_years = ['10년', '6년', '5년', '3년', '2년', '1년']

# 딕셔너리 초기화
ssd = dict()
ssd_pci = dict()

for col in header:
    ssd[col] = []

1198


In [63]:
# 크롤링 참고 (크롤링, 딕셔너리로 저장)

# chrome_options = Options()

user_agent ="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
chrome_options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(options=chrome_options)


# 크롤링 시작
ntime = 1
start = time.time()

for seq in seq_df.seq:
    url = f'https://prod.danawa.com/info/?pcode={seq}&cate={category}'
    driver.get(url)
    driver.implicitly_wait(10)

    time.sleep(waiting_sec)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    spec_tbl = soup.select("#productDescriptionArea > div > div.prod_spec > table > tbody")
    spec_tbl_tit = soup.select("#productDescriptionArea > div > div.prod_spec > table > tbody > tr > th.tit")
    spec_tbl_dsc = soup.select("#productDescriptionArea > div > div.prod_spec > table > tbody > tr > td.dsc")
    name = soup.select_one("#blog_content > div.summary_info > div.top_summary > h3 > span")
    price = soup.select_one("#blog_content > div.summary_info > div.detail_summary > div.summary_left > div.lowest_area > div.lowest_top > div.row.lowest_price > span.lwst_prc > a > em")
    image_url = soup.select_one("#baseImage")
    
    tbl = zip(spec_tbl_tit, spec_tbl_dsc)

    info = dict()
    info['제품명'] = name.get_text()
    info['최저가'] = price.get_text() if price else None
    info['링크'] = url
    info['이미지'] = image_url['src']
    for tit, dsc in tbl:
        if tit.get_text() in ['안전확인인증','적합성평가인증']:
            continue
        text = dsc.get_text()
        text = text.replace('\n','')
        text = text.replace('\t','')
        text = text.replace('(제조사 웹사이트 바로가기)','')
        info[tit.get_text()] = text.strip()
    
    # pprint.pprint(info)
    
   
    m2i_bit = 0
    m2f_bit = 0
    wl_bit = 0
    wls = 0
    
    
    
    ssd['name'].append(name.get_text()[:name.get_text().index('(') if '(' in name.get_text() else None].rstrip())
    ssd['price'].append(price.get_text().replace(',','') if price else None)
    ssd['link'].append(url)
    ssd['company'].append(info.get('제조회사'))
    ssd['product_seq'].append(seq)
    ssd['image'].append(image_url.get('src'))
    
    ssd['form_factor'].append(info.get('폼팩터'))
    ssd['interface'].append(info.get('인터페이스'))
    ssd['protocol'].append(info.get('프로토콜'))
    ssd['volume'].append(info.get('용량'))
    
    ssd['memory_type'].append(info.get('메모리 타입'))
    ssd['nand'].append(info.get('낸드 구조'))
    
    ssd['ram_mounted'].append(info.get('RAM 탑재'))
    ssd['ram_type'].append(info.get('RAM 타입'))
    
    ssd['sequential_read'].append(info.get('순차읽기'))
    ssd['sequential_write'].append(info.get('순차쓰기'))
    
    ssd['read_iops'].append(info.get('읽기IOPS'))
    ssd['write_iops'].append(info.get('쓰기IOPS'))

    ssd['heatsink'].append(1 if info.get('히트싱크')=='방열판' else 0)
    
    
    ssd['rgbled'].append(1 if info.get('RGB/LED') else 0)
    
    for year in as_years:
        if year in info.keys():
            ssd['as_year'].append(year)
            break
    else:
        ssd['as_year'].append(None)
    
    support_bit = 0
    for i in range(len(support_bitmask)):
        if support_bitmask[i] in info.keys(): support_bit |= 1 << i
    ssd['support_option'].append(support_bit)

                          
    ssd['reg_date'].append(re.sub('(년| |월)', '', info.get('등록년월')))
    ssd['bookmark'].append(info.get('북마크'))
    
    recent = time.time()
    
    remain = int(recent-start)//ntime*(len_seq-ntime)
    
    print(f'\r{ntime} / {len_seq} : {int(ntime/len_seq*100)}% / 남은시간: {remain//3600}:{remain%3600//60}:{remain%60}',end='')
    
    ntime+=1
    


end = time.time()

print(f"\n걸린 시간: {end-start:.2f}sec")
driver.quit()
# 크롤링 끝)

1198 / 1198 : 100% / 남은시간: 0:0:00
걸린 시간: 6542.45sec


In [64]:
ssd_df = pd.DataFrame(ssd)
ssd_df.tail()

,name,price,link,company,product_seq,image,form_factor,interface,protocol,volume,...,sequential_read,sequential_write,read_iops,write_iops,heatsink,rgbled,as_year,support_option,reg_date,bookmark
1412,라이트컴 COMS A3992 하드케이스,1700,https://prod.danawa.com/info/?pcode=1649117&ca...,라이트컴 COMS,1649117,//img.danawa.com/prod_img/500000/117/649/img/1...,None,None,None,None,...,None,None,None,None,0,0,None,0,201204,None
1413,라이트컴 COMS A3993 하드케이스,1700,https://prod.danawa.com/info/?pcode=1649110&ca...,라이트컴 COMS,1649110,//img.danawa.com/prod_img/500000/110/649/img/1...,None,None,None,None,...,None,None,None,None,0,0,None,0,201204,None
1414,명정보기술 MITSF200,223600,https://prod.danawa.com/info/?pcode=1634506&ca...,명정보기술,1634506,//img.danawa.com/prod_img/500000/506/634/img/1...,6.4cm(2.5형),SATA3(6Gb/s),None,128GB,...,550MB/s,520MB/s,None,None,0,0,None,1,201203,None
1415,앤디코 AIO mSATA to SATA 브릿지,23000,https://prod.danawa.com/info/?pcode=1574436&ca...,앤디코,1574436,//img.danawa.com/prod_img/500000/436/574/img/1...,None,None,None,None,...,None,None,None,None,0,0,None,0,201201,None
1416,EKMEMORY RealSSD C300,390000,https://prod.danawa.com/info/?pcode=1187403&ca...,EKMEMORY,1187403,//img.danawa.com/prod_img/500000/403/187/img/1...,None,SATA3(6Gb/s),None,128GB,...,355MB/s,145MB/s,None,None,0,0,None,2,201009,None


In [66]:
pd.DataFrame(ssd).to_csv("ssd.csv")

In [62]:
# name.get_text()[:name.get_text().index('(') if '(' in name.get_text() else None].rstrip()

'KINGMAX 3.5 브라켓'

In [9]:
ssd_df = pd.read_csv("./ssd.csv", index_col=0)

In [10]:
ssd_df.tail()

,name,price,link,company,product_seq,image,form_factor,interface,protocol,volume,...,sequential_read,sequential_write,read_iops,write_iops,heatsink,rgbled,as_year,support_option,reg_date,bookmark
1412,라이트컴 COMS A3992 하드케이스,1700.0,https://prod.danawa.com/info/?pcode=1649117&ca...,라이트컴 COMS,1649117,//img.danawa.com/prod_img/500000/117/649/img/1...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,NaN,0,201204,NaN
1413,라이트컴 COMS A3993 하드케이스,1700.0,https://prod.danawa.com/info/?pcode=1649110&ca...,라이트컴 COMS,1649110,//img.danawa.com/prod_img/500000/110/649/img/1...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,NaN,0,201204,NaN
1414,명정보기술 MITSF200,223600.0,https://prod.danawa.com/info/?pcode=1634506&ca...,명정보기술,1634506,//img.danawa.com/prod_img/500000/506/634/img/1...,6.4cm(2.5형),SATA3(6Gb/s),NaN,128GB,...,550MB/s,520MB/s,NaN,NaN,0,0,NaN,1,201203,NaN
1415,앤디코 AIO mSATA to SATA 브릿지,23000.0,https://prod.danawa.com/info/?pcode=1574436&ca...,앤디코,1574436,//img.danawa.com/prod_img/500000/436/574/img/1...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,NaN,0,201201,NaN
1416,EKMEMORY RealSSD C300,390000.0,https://prod.danawa.com/info/?pcode=1187403&ca...,EKMEMORY,1187403,//img.danawa.com/prod_img/500000/403/187/img/1...,NaN,SATA3(6Gb/s),NaN,128GB,...,355MB/s,145MB/s,NaN,NaN,0,0,NaN,2,201009,NaN


In [13]:
def pre_sequential_wr(x):
    x = str(x).replace('MB/s','').strip()
    return x
def pre_iops_wr(x):
    x = str(x).replace('최대','').replace('K','').strip()
    return x

def pre_year(x):
    x = str(x).replace('년', '').strip()
    return x

ssd_df.sequential_read = ssd_df.sequential_read.apply(lambda x: pre_sequential_wr(x))
ssd_df.sequential_write = ssd_df.sequential_write.apply(lambda x: pre_sequential_wr(x))

ssd_df.read_iops = ssd_df.read_iops.apply(lambda x: pre_iops_wr(x))
ssd_df.write_iops = ssd_df.write_iops.apply(lambda x: pre_iops_wr(x))

ssd_df.as_year = ssd_df.as_year.apply(lambda x: pre_year(x))

ssd_df.tail()

,name,price,link,company,product_seq,image,form_factor,interface,protocol,volume,...,sequential_read,sequential_write,read_iops,write_iops,heatsink,rgbled,as_year,support_option,reg_date,bookmark
1412,라이트컴 COMS A3992 하드케이스,1700.0,https://prod.danawa.com/info/?pcode=1649117&ca...,라이트컴 COMS,1649117,//img.danawa.com/prod_img/500000/117/649/img/1...,NaN,NaN,NaN,NaN,...,nan,nan,nan,nan,0,0,nan,0,201204,NaN
1413,라이트컴 COMS A3993 하드케이스,1700.0,https://prod.danawa.com/info/?pcode=1649110&ca...,라이트컴 COMS,1649110,//img.danawa.com/prod_img/500000/110/649/img/1...,NaN,NaN,NaN,NaN,...,nan,nan,nan,nan,0,0,nan,0,201204,NaN
1414,명정보기술 MITSF200,223600.0,https://prod.danawa.com/info/?pcode=1634506&ca...,명정보기술,1634506,//img.danawa.com/prod_img/500000/506/634/img/1...,6.4cm(2.5형),SATA3(6Gb/s),NaN,128GB,...,550,520,nan,nan,0,0,nan,1,201203,NaN
1415,앤디코 AIO mSATA to SATA 브릿지,23000.0,https://prod.danawa.com/info/?pcode=1574436&ca...,앤디코,1574436,//img.danawa.com/prod_img/500000/436/574/img/1...,NaN,NaN,NaN,NaN,...,nan,nan,nan,nan,0,0,nan,0,201201,NaN
1416,EKMEMORY RealSSD C300,390000.0,https://prod.danawa.com/info/?pcode=1187403&ca...,EKMEMORY,1187403,//img.danawa.com/prod_img/500000/403/187/img/1...,NaN,SATA3(6Gb/s),NaN,128GB,...,355,145,nan,nan,0,0,nan,2,201009,NaN


In [14]:
ssd_df.to_csv("ssd_final.csv", index=None)